In [ ]:
import sys
sys.path.append('/Users/vdk/Software/code/')
import h5py
import pandas as pd
from astropy.table import Table, vstack
import numpy as np
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt
import glob
from python_modules.muonpipe import usefull_func
import seaborn as sns
from ctapipe.image import tailcuts_clean
import scipy.stats as stats
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    'k'
]
filename = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry

## Choose event_id of good muon events for further processing dl1 .h5 files on the cluster

In [ ]:
fits_file = '/Users/vdk/muons2024/data/20240310fits/muons_LST-1.Run17043.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99
event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]))

min_impact = min(df_good_data['impact_parameter'])
max_impact = max(df_good_data['impact_parameter']) 

filename_event_ids = '/Users/vdk/muons2024/data/17043_muons_08complet.txt'
#np.save(filename_event_ids, event_ids)

df_real_data = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]

In [ ]:
file_path = '/Users/vdk/muons2024/data/dl1_LST-1.Run17043.0001.h5'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    print(f"dataset {dataset}")
    print(f"dataset type {type(dataset)}")
    # Read the data
    data = dataset[:]
    print(f"data {data}")
    print(f"data type {type(data)}")
    
    # Print some information about the data
    # print(f'Dataset shape: {dataset.shape}')
    # print(f'Dataset dtype: {dataset.dtype}')
    # print('Data:', data)

    # If you want to read a specific part of the data, you can use slicing
    # For example, to read the first 10 entries:
    partial_data = dataset[:10]
#    print('First 10 entries:', partial_data)

In [ ]:
file_path = '/Users/vdk/muons2024/data/dl1_LST-1.Run17043.h5'
with h5py.File(file_path, 'r') as hdf:
    # Function to recursively print all group and dataset names
    def print_name(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}")
        elif isinstance(obj, h5py.Group):
            print(f"Group: {name}")

    # Iterate through the file and print all table names (datasets)
    hdf.visititems(print_name)


# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/parameters/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    # Print some information about the data
    print(f'Dataset shape: {dataset.shape}')
    print(f'Dataset dtype: {dataset.dtype}')
    print('Data:', data)

    # If you want to read a specific part of the data, you can use slicing
    # For example, to read the first 10 entries:
    partial_data = dataset[:10]
    print('First 10 entries:', partial_data)

In [ ]:
for event_type in data:
    if event_type[-1] == 16:
        print(event_type[-1])

In [ ]:
dataset['dl1/event/telescope/parameters/LST_LSTCam']

In [ ]:
import h5py
import glob
import numpy as np

filename_event_ids = '/fefs/aswg/workspace/vadym.voitsekhovskyi/2024muons/lst_study/real_data/17043/17043_muons_08complet.txt.npy'
event_ids = np.load(filename_event_ids)


file_path = glob.glob('/fefs/aswg/data/real/DL1/20240310/v0.10/tailcut84/dl1_LST-1.Run17043.*.h5')

# Open the .h5 file
for filename in file_path:
    with h5py.File(file_path, 'r') as f:
        # Access the dataset
        dataset = f['dl1/event/telescope/image/LST_LSTCam']
        
        # Read the data
        data = dataset[:]
        
        for event_record in data:
            if event_record[1] in event_ids:
                filename_image = f'/fefs/aswg/workspace/vadym.voitsekhovskyi/2024muons/lst_study/real_data/17043/dl1_image_event{event_record[1]}.txt'
                filename_peak_time = f'/fefs/aswg/workspace/vadym.voitsekhovskyi/2024muons/lst_study/real_data/17043/dl1__peak_time_event{event_record[1]}.txt'
                filename_image_mask = f'/fefs/aswg/workspace/vadym.voitsekhovskyi/2024muons/lst_study/real_data/17043/dl1__image_mask_event{event_record[1]}.txt'
                np.save(filename_image, event_record[2])
                np.save(filename_image_mask, event_record[4])
                np.save(filename_peak_time, event_record[3])
        

## Simulations Loading 

### LocalPeakWindowSum

Its a standard way how lstchain is processing muon rings in simulations. But for muons in the data lstchain uses GlobalPeakWindowSum

In [ ]:
local_fits_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/local_peak_window_sum/muon_table_run1.fits'
local_dl1_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/local_peak_window_sum/dl1_run101_muon.h5'

dat = Table.read(local_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_local = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

del(df_good_data)

local_sim_dl1_image = []
local_sim_dl1_peak_time = []
local_sim_dl1_image_mask = []
with h5py.File(local_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            local_sim_dl1_image.append(event_record[2])
            local_sim_dl1_peak_time.append(event_record[3])
            local_sim_dl1_image_mask.append(event_record[4])
        if len(local_sim_dl1_image) > 507:
            break

In [ ]:
filename = '/Users/vdk/muons2024/lapalma_simulations/simtel/zenith10/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 5000)
for i,event in enumerate(local_sim_dl1_image[:10]):
        plt.figure(figsize = (12,12))
        camgeom = source.subarray.tel[1].camera.geometry
        #title="CT{}, run {} cog {} fit {} width {}".format(1,event.index.obs_id, round(event.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title='1')
        disp.image = event #.sum(axis=1)
        #disp.image = j.r0.tel[1].waveform[0].sum(axis=1)        
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
        plt.show()

### GlobalPeakWindowSum

I specially used other config file with GlobalPeakWindowSum ImageExtractor for the same simtel file to check the difference between different charge extractors

In [ ]:
global_fits_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/global_peak_window_sum/tables/muon_table_run1.fits'
global_dl1_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/global_peak_window_sum/dl1_run101_muon.h5'

dat = Table.read(global_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

ring_completeness_cut = 0.8
ring_containment_cut = 0.99
event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

global_sim_dl1_image = []
global_sim_dl1_peak_time = []
global_sim_dl1_image_mask = []
with h5py.File(global_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            global_sim_dl1_image.append(event_record[2])
            global_sim_dl1_peak_time.append(event_record[3])
            global_sim_dl1_image_mask.append(event_record[4])
        if len(global_sim_dl1_image) > 507:
            break


In [ ]:
for i,event in enumerate(global_sim_dl1_image[:10]):
        plt.figure(figsize = (12,12))
        camgeom = source.subarray.tel[1].camera.geometry
        #title="CT{}, run {} cog {} fit {} width {}".format(1,event.index.obs_id, round(event.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title='1')
        disp.image = event #.sum(axis=1)
        #disp.image = j.r0.tel[1].waveform[0].sum(axis=1)        
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
        plt.show()

### NSB Tuned simulation

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb/muon_table_run1.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_nsb1data_tuned = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

nsb_tuned_dl1_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb//dl1_run101_muon.h5'
nsb1data_tuned_sim_dl1_image = []
nsb1data_tuned_sim_dl1_peak_time = []
nsb1data_tuned_sim_dl1_image_mask = []
with h5py.File(nsb_tuned_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsb1data_tuned_sim_dl1_image.append(event_record[2])
            nsb1data_tuned_sim_dl1_peak_time.append(event_record[3])
            nsb1data_tuned_sim_dl1_image_mask.append(event_record[4])
        if len(nsb1data_tuned_sim_dl1_image) > 507:
            break

fits_file = '/Users/vdk/muons2024/data/17043/nsb_tuning_dl1/muon_fits_tuned_2.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_nsb3data_tuned = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

nsb_tuned_dl1_file = '/Users/vdk/muons2024/data/17043/nsb_tuning_dl1/tuned.h5'
nsbDL1data_tuned_sim_dl1_image = []
nsbDL1data_tuned_sim_dl1_peak_time = []
nsbDL1data_tuned_sim_dl1_image_mask = []
with h5py.File(nsb_tuned_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsbDL1data_tuned_sim_dl1_image.append(event_record[2])
            nsbDL1data_tuned_sim_dl1_peak_time.append(event_record[3])
            nsbDL1data_tuned_sim_dl1_image_mask.append(event_record[4])
        if len(nsbDL1data_tuned_sim_dl1_image) > 507:
            break

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb_local_peak/muon_table_run1.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_nsb3data_tuned = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

nsb_tuned_local_peak_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb_local_peak/dl1_run101_muon.h5'
nsb_tuned_local_peak_dl1_image = []
nsb_tuned_local_peak_peak_time = []
nsb_tuned_local_peak_image_mask = []
with h5py.File(nsb_tuned_local_peak_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsb_tuned_local_peak_dl1_image.append(event_record[2])
            nsb_tuned_local_peak_peak_time.append(event_record[3])
            nsb_tuned_local_peak_image_mask.append(event_record[4])
        if len(nsb_tuned_local_peak_dl1_image) > 507:
            break

In [ ]:
nsb_tuned_dl1_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb//dl1_run101_muon.h5'
nsb_tuned_sim_dl1_image = []
nsb_tuned_sim_dl1_peak_time = []
nsb_tuned_sim_dl1_image_mask = []
with h5py.File(nsb_tuned_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsb_tuned_sim_dl1_image.append(event_record[2])
            nsb_tuned_sim_dl1_peak_time.append(event_record[3])
            nsb_tuned_sim_dl1_image_mask.append(event_record[4])
        if len(nsb_tuned_sim_dl1_image) > 507:
            break


### Triple tuning

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/triple_tuning/muon_table_run1.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_nsb3data_tuned = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

nsb_tuned_double_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb_local_peak/dl1_run101_muon.h5'
nsb_tuned_double_dl1_image = []
nsb_tuned_double_peak_time = []
nsb_tuned_double_image_mask = []
with h5py.File(nsb_tuned_double_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            nsb_tuned_double_dl1_image.append(event_record[2])
            nsb_tuned_double_peak_time.append(event_record[3])
            nsb_tuned_double_image_mask.append(event_record[4])
        if len(nsb_tuned_double_dl1_image) == 507:
            break

### Mirror degraded relfection tuned sim

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection_1.2/muon_table_run1.fits'


dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_reflection_09408 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))



In [ ]:
relection_dl1_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection_1.2/dl1_run101_muon.h5'
reflection09408_sim_dl1_image = []
reflection09408_sim_dl1_peak_time = []
reflection09408_sim_dl1_image_mask = []
with h5py.File(nsb_tuned_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            reflection09408_sim_dl1_image.append(event_record[2])
            reflection09408_sim_dl1_peak_time.append(event_record[3])
            reflection09408_sim_dl1_image_mask.append(event_record[4])
        if len(reflection09408_sim_dl1_image) > 507:
            break

len(reflection09408_sim_dl1_image)

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection_1.1/muon_table_run1.fits'


dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_reflection_08624 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))



In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/global_peak_window_sum/degraded_reflection-9016/muon_table_run1.fits'


dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_sim_data_reflection_09016 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))



## Data Loading

In [ ]:
image_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_image/*')
mask_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_mask/*')
peak_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_peak/*')

data_dl1_images = []
data_dl1_mask = []
data_dl1_peak = []

for file in image_folder:
    tmp_result = np.load(file)
    data_dl1_images.append(tmp_result)

for file in mask_folder:
    tmp_result = np.load(file)
    data_dl1_mask.append(tmp_result)

for file in peak_folder:
    tmp_result = np.load(file)
    data_dl1_peak.append(tmp_result)

In [ ]:
for i,event in enumerate(data_dl1_images[:50]):
        plt.figure(figsize = (12,12))
        camgeom = source.subarray.tel[1].camera.geometry
        #title="CT{}, run {} cog {} fit {} width {}".format(1,event.index.obs_id, round(event.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title='1')
        disp.image = event #.sum(axis=1)
        #disp.image = j.r0.tel[1].waveform[0].sum(axis=1)        
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
        plt.show()

In [ ]:
muon_files = glob.glob('/Users/vdk/muons2024/data/20240310fits/*')

dat = Table.read('{}'.format(muon_files[0]), format='fits')

for muon_file in muon_files:
    dat2 = Table.read(muon_file, format='fits')
    dat2['good_ring'] = dat2['good_ring'].astype(bool)
    dat = vstack([dat, dat2])

df = dat.to_pandas()
df_real_data_stacked = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500) &
                      (df['ring_completeness'] > ring_completeness_cut) &
                      (df['ring_containment'] > ring_containment_cut)]


df_real_data_stacked

### High nsb subrun

In [ ]:
fits_file = '/Users/vdk/muons2024/data/16876/muons_LST-1.Run16876.0054.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.49
ring_containment_cut = 0.49

df_real_data_high_nsb = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

real_data_high_nsb_file = '/Users/vdk/muons2024/data/16876/dl1_LST-1.Run16876.0054.h5'
real_data_high_nsb_dl1_image = []
real_data_high_nsb_dl1_peak_time = []
real_data_high_nsb_dl1_image_mask = []
with h5py.File(real_data_high_nsb_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        #if event_record[1] in event_ids:
        if True:
            real_data_high_nsb_dl1_image.append(event_record[2])
            real_data_high_nsb_dl1_peak_time.append(event_record[3])
            real_data_high_nsb_dl1_image_mask.append(event_record[4])
        if len(real_data_high_nsb_dl1_image) > 507:
            break

### NSB Tuning on waveform but with proper reference original nsb

In [ ]:
fits_file = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb_changed_original_nsb/muon_table_run1.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_real_data_proper_nsb_tune = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

real_data_proper_nsb_tune = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/0.44nsb_changed_original_nsb/dl1_run101_muon.h5'
real_data_proper_nsb_tune_dl1_image = []
real_data_proper_nsb_tune_peak_time = []
real_data_proper_nsb_tune_image_mask = []
with h5py.File(real_data_proper_nsb_tune, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        #if event_record[1] in event_ids:
        if True:
            real_data_proper_nsb_tune_dl1_image.append(event_record[2])
            real_data_proper_nsb_tune_peak_time.append(event_record[3])
            real_data_proper_nsb_tune_image_mask.append(event_record[4])
        if len(real_data_proper_nsb_tune_dl1_image) > 507:
            break

### Particular data subrun

In [ ]:


fits_file = '/Users/vdk/muons2024/data/17043/muons_LST-1.Run17043.0020.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

df_real_data_high_nsb = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

data_subrun_file = '/Users/vdk/muons2024/data/17043/dl1_LST-1.Run17043.0020.h5'
data_subrun_dl1_image = []
data_subrun_dl1_peak_time = []
data_subrun_dl1_image_mask = []
with h5py.File(real_data_high_nsb_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        #if event_record[1] in event_ids:
        if True:
            data_subrun_dl1_image.append(event_record[2])
            data_subrun_dl1_peak_time.append(event_record[3])
            data_subrun_dl1_image_mask.append(event_record[4])
        if len(data_subrun_dl1_image) > 507:
            break

## Histograms for fits files

In [ ]:
plt.figure(figsize=(9,6))
usefull_func.hist_wo_outliers(df_real_data['ring_radius'], label = 'Data', density=True, bin_number = 15, alpha=0.5)
usefull_func.hist_wo_outliers(df_sim_data_global['ring_radius'], bin_number = 15, label = 'Simulations NSB 0.24GHz', density=True, alpha=0.5)
#usefull_func.hist_wo_outliers(df_sim_data_local['ring_radius'], bin_number = 30, label = 'Simulations LocalPeak', density=True, alpha=0.5)
usefull_func.hist_wo_outliers(df_sim_data_nsb_tuned['ring_radius'], bin_number = 15, label = 'Simulations NSB tuned', density=True, alpha=0.6)

plt.legend()
plt.grid(alpha=0.5)
plt.xlabel("Ring radius [deg]")
plt.ylabel("Probability density")
#plt.title("Histogram for ring radius")
plt.savefig('/Users/vdk/muons2024/images/PlanPaper/hist_radius.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
len(df_real_data['ring_radius'])

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(df_real_data['ring_size'], bins = 20, alpha = 0.6, label = 'Data', density=True)
plt.hist(df_sim_data_global['ring_size'], bins = 20, alpha = 0.6, label = 'Simulations NSB 0.24 GHz', density=True)
plt.hist(df_sim_data_nsb_tuned['ring_size'], bins = 20, alpha = 0.6, label = 'Simulations NSB tuned', density=True)
plt.legend()
plt.grid(alpha=0.5)
plt.xlabel("Total intensity in ring in photoelectrons")
plt.ylabel("Probability density")
#plt.title("Histogram for ring size")
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/hist_size.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(9,6))
usefull_func.hist_wo_outliers(df_real_data_stacked['ring_width'], label = 'Data', density=True, bin_number = 20, outlier_value=0.3, alpha=0.6)
usefull_func.hist_wo_outliers(df_sim_data_local['ring_width'], label = 'Simulations LocalPeak', density=True, bin_number = 20, outlier_value=0.3, alpha=0.6)
usefull_func.hist_wo_outliers(df_sim_data_global['ring_width'], bin_number = 20, label = 'Simulations GlobalPeak', density=True, outlier_value=0.3, alpha=0.6)
plt.legend()
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
plt.xlabel("Ring width [deg]")
plt.ylabel("Probability density")
#plt.title("Histogram for ring width")
plt.savefig('/Users/vdk/muons2024/images/PlanPaper/hist_width.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(df_real_data_stacked['num_pixels_in_ring'], bins = 20, alpha = 0.6, label = 'Data', density=True)
plt.hist(df_sim_data_local['num_pixels_in_ring'], bins = 20, alpha = 0.6, label = 'Simulations LocalPeak', density=True)
plt.hist(df_sim_data_global['num_pixels_in_ring'], bins = 20, alpha = 0.6, label = 'Simulations GlobalPeak', density=True)
plt.legend()
plt.grid(alpha=0.5)
plt.xlabel("Pixels inside the integration area around the ring")
plt.ylabel("Probability density")
#plt.title("Histogram for number of pixels in ring")
plt.savefig('/Users/vdk/muons2024/images/PlanPaper/hist_pixel_number.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(df_real_data_stacked['impact_parameter'], bins = 20, alpha = 0.6, label = 'Data', density=True)
plt.hist(df_sim_data_local['impact_parameter'], bins = 20, alpha = 0.6, label = 'Simulations LocalPeak', density=True)
plt.hist(df_sim_data_global['impact_parameter'], bins = 20, alpha = 0.6, label = 'Simulations GlobalPeak', density=True)
plt.legend()
plt.grid(alpha=0.5)
plt.xlabel("Impact parameter [m]")
plt.ylabel("Probability density")
#plt.title("Histogram for impact parameter")
plt.savefig('/Users/vdk/muons2024/images/PlanPaper/hist_impact_param.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(9,6))
usefull_func.hist_wo_outliers(df_real_data_stacked['muon_efficiency'], label = 'Data', density=True, bin_number = 30, outlier_value=0.3, alpha = 0.6)
usefull_func.hist_wo_outliers(df_sim_data_global['muon_efficiency'], label = 'Simulations 0.24 GHz', density=True, bin_number = 30, outlier_value=0.3, alpha=0.6)
usefull_func.hist_wo_outliers(df_sim_data_nsb1data_tuned['muon_efficiency'], label = 'Simulations NSB tuned to data', density=True, bin_number = 30, outlier_value=0.3, alpha=0.6)

plt.legend()
plt.grid(alpha=0.5)
plt.xlabel("Muon efficiency")
plt.ylabel("Probability density")
#plt.title("Histogram for muon efficiency")
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/hist_efficiency.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(9,6))
usefull_func.hist_wo_outliers(df_real_data_stacked['size_outside'], label = 'Data', density=True, bin_number = 20, outlier_value=20, show_outliers=True)
usefull_func.hist_wo_outliers(df_sim_data_local['size_outside'], label = 'Simulations LocalPeak', density=True, bin_number = 20, outlier_value=20, show_outliers=True)
usefull_func.hist_wo_outliers(df_sim_data_global['size_outside'], label = 'Simulations GlobalPeak', density=True, bin_number = 20, outlier_value=20, show_outliers=True)
plt.legend()
plt.grid(alpha=0.5)
plt.xlabel("Intensity outside the muon ting in photoelectrons")
plt.ylabel("Probability density")
plt.title("Histogram for size outside ")

### Charge in outer ring

**Mask to select pixels just outside the ring that will be integrated to obtain the ring's intensity:**

```python
dist_mask_2 = np.logical_and(~dist_mask,
                                np.abs(dist - muonringparam.radius) <
                                muonringparam.radius *
                                (params['ring_integration_width'] + params['outer_ring_width']))
pix_ring_2 = image[dist_mask_2]
```

**Where:**
```python
        'ring_integration_width': 0.25,  # +/- integration range along ring radius,
                                         # in fraction of ring radius (was 0.4 until 20200326)
        'outer_ring_width': 0.2,  # in fraction of ring radius, width of ring just outside the
                                  # integrated muon ring, used to check pedestal bias
```

**This is just mean charge per pixel in pixels just around the ring (on the outer side):**

```python
mean_pixel_charge_around_ring = np.sum(pix_ring_2) / len(pix_ring_2)
```

In [ ]:
plt.figure(figsize=(9,6))
usefull_func.hist_wo_outliers(df_real_data_stacked['mean_pixel_charge_around_ring'], label = 'Data', density=True, bin_number = 20, outlier_value=50, color = colors[0], alpha = 0.6)
usefull_func.hist_wo_outliers(df_sim_data_local['mean_pixel_charge_around_ring'], label = 'Simulations LocalPeak', density=True, bin_number = 20, outlier_value=50, color = colors[1], alpha = 0.6)
usefull_func.hist_wo_outliers(df_sim_data_global['mean_pixel_charge_around_ring'], label = 'Simulations GlobalPeak', density=True, bin_number = 20, outlier_value=50, color = colors[2], alpha = 0.6)
usefull_func.hist_wo_outliers(df_sim_data_nsb_tuned['mean_pixel_charge_around_ring'], label = 'Simulations NSB Tuned', density=True, bin_number = 20, outlier_value=50, color = colors[3], alpha = 0.6)
plt.legend()
plt.grid(alpha=0.5)
plt.xlabel("Average pixel charge in pixels surrounding the outer part of the ring")
plt.ylabel("Probability density")
#plt.title("Histogram for mean pixel charge around ring ")
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/hist_pixel_charge_around_ring.pdf', dpi=200, format='pdf', bbox_inches='tight')

## Additional cleaning


In [ ]:
new_data_mask = []
for image in data_dl1_images:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_data_mask.append(mask)
    
new_high_nsb_data_mask = []
for image in real_data_high_nsb_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_high_nsb_data_mask.append(mask)
    
new_data_proper_nsb_tune_mask = []
for image in real_data_proper_nsb_tune_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_data_proper_nsb_tune_mask.append(mask)
    
new_sim_mask = []
for image in nsb1data_tuned_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask.append(mask)

new_sim_mask_global = []
for image in global_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask_global.append(mask)
    
new_data_subrun_mask = []
for image in data_subrun_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_data_subrun_mask.append(mask)
    
new_local_tune_subrun_mask = []
for image in nsb_tuned_local_peak_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_local_tune_subrun_mask.append(mask)
    


In [ ]:
new_data_proper_nsb_tune_mask

## DL1 images processing

### Charge per pixel

In [ ]:
bin_quantity = 1000
plt.figure(figsize=(12,9))
flattened_sim_nsb = np.array(nsb1data_tuned_sim_dl1_image[:2000]).flatten()
flattened_sim_local = np.array(local_sim_dl1_image).flatten()
flattened_sim_global = np.array(global_sim_dl1_image).flatten()
flattened_data = np.array(data_dl1_images).flatten()
usefull_func.hist_wo_outliers(flattened_data, label = 'Data', density=True, mean_line=True, show_outliers=True, outlier_value=30, xlabel='Charge per pixel', ylabel='Probability density', bin_number=bin_quantity, color = colors[0], alpha = 0.6)
#usefull_func.hist_wo_outliers(flattened_sim_local, label = 'Simulations Local Peak', density=True, mean_line=True, show_outliers=True, outlier_value=30, xlabel='Charge per pixel', ylabel='Probability density', bin_number=bin_quantity, color = colors[1], alpha = 0.6)
usefull_func.hist_wo_outliers(flattened_sim_global, label = 'Simulations standard NSB=0.24 GHz', density=True, mean_line=True, show_outliers=True, outlier_value=30, xlabel='Charge per pixel', ylabel='Probability density', bin_number=bin_quantity, color = colors[2], alpha = 0.6)
usefull_func.hist_wo_outliers(flattened_sim_nsb, label = 'Simulations NSB Tuned for data', density=True, mean_line=True, show_outliers=True, outlier_value=30, xlabel='Charge per pixel', ylabel='Probability density', bin_number=bin_quantity, color = colors[3], alpha = 0.6)

plt.legend()

In [ ]:
new_data_proper_nsb_tune_mask

In [ ]:
flattened_sim_local = np.array(local_sim_dl1_image).flatten()
flattened_sim_global = np.array(global_sim_dl1_image).flatten()
flattened_sim_nsb = np.array(nsb1data_tuned_sim_dl1_image[:2000]).flatten()
flattened_sim_nsb_local = np.array(nsb_tuned_local_peak_dl1_image).flatten()
flattened_data = np.array(data_dl1_images).flatten()
flattened_sim_nsbDL1data = np.array(nsbDL1data_tuned_sim_dl1_image[:600]).flatten()
flattened_data_high_nsb = np.array(real_data_high_nsb_dl1_image).flatten()
flattened_data_subrun = np.array(data_subrun_dl1_image).flatten()
flattened_data_proper_nsb = np.array(real_data_proper_nsb_tune_dl1_image).flatten()
# flattened_sim_nsb10data = np.array(nsb10data_tuned_sim_dl1_image[:600]).flatten()

data_masks_flat = np.array(data_dl1_mask).flatten()
data_masks_flat = np.array(new_data_mask).flatten()
local_sim_masks_flat = np.array(local_sim_dl1_image_mask).flatten()
data_high_nsb_mask_flat = np.array(real_data_high_nsb_dl1_image_mask).flatten()
data_high_nsb_mask_flat = np.array(new_high_nsb_data_mask).flatten()


data_subrun_mask_flat = np.array(new_data_subrun_mask).flatten()

global_sim_masks_flat = np.array(global_sim_dl1_image_mask).flatten()
global_sim_masks_flat = np.array(new_sim_mask_global).flatten()

nsb_tuned_sim_masks_flat = np.array(nsb_tuned_sim_dl1_image_mask).flatten()
nsb_tuned_sim_masks_flat = np.array(new_sim_mask).flatten()
nsb_tuned_local_sim_masks_flat = np.array(new_local_tune_subrun_mask).flatten()

nsbDL1data_tuned_sim_masks_flat = np.array(nsbDL1data_tuned_sim_dl1_image_mask).flatten()
# nsb10data_tuned_sim_masks_flat = np.array(nsb10data_tuned_sim_dl1_image_mask).flatten()

real_data_proper_nsb_tune_mask_flat = np.array(new_data_proper_nsb_tune_mask).flatten()

# Separate pixel values based on the mask
data_pixels_with_mask_1 = flattened_data[data_masks_flat == 1]
data_pixels_with_mask_0 = flattened_data[data_masks_flat == 0]

data_proper_nsb_tune_pixels_with_mask_1 = flattened_data_proper_nsb[real_data_proper_nsb_tune_mask_flat == 1]
data_proper_nsb_tune_pixels_with_mask_0 = flattened_data_proper_nsb[real_data_proper_nsb_tune_mask_flat == 0]

data_high_nsb_pixels_with_mask_1 = flattened_data_high_nsb[data_high_nsb_mask_flat == 1]
data_high_nsb_pixels_with_mask_0 = flattened_data_high_nsb[data_high_nsb_mask_flat == 0]

data_subrun_pixels_with_mask_1 = flattened_data_subrun[data_subrun_mask_flat == 1]
data_subrun_pixels_with_mask_0 = flattened_data_subrun[data_subrun_mask_flat == 0]

local_sim_pixels_with_mask_1 = flattened_sim_local[local_sim_masks_flat == 1]
local_sim_pixels_with_mask_0 = flattened_sim_local[local_sim_masks_flat == 0]

global_sim_pixels_with_mask_1 = flattened_sim_global[global_sim_masks_flat == 1]
global_sim_pixels_with_mask_0 = flattened_sim_global[global_sim_masks_flat == 0]

nsb_tuned_sim_pixels_with_mask_1 = flattened_sim_nsb[nsb_tuned_sim_masks_flat == 1]
nsb_tuned_sim_pixels_with_mask_0 = flattened_sim_nsb[nsb_tuned_sim_masks_flat == 0] 

nsb_tuned_local_pixels_with_mask_1 = flattened_sim_nsb_local[nsb_tuned_local_sim_masks_flat == 1]
nsb_tuned_local_pixels_with_mask_0 = flattened_sim_nsb_local[nsb_tuned_local_sim_masks_flat == 0] 

nsbDL1data_tuned_sim_pixels_with_mask_1 = flattened_sim_nsbDL1data[nsbDL1data_tuned_sim_masks_flat == 1]
nsbDL1data_tuned_sim_pixels_with_mask_0 = flattened_sim_nsbDL1data[nsbDL1data_tuned_sim_masks_flat == 0]

# nsb10data_tuned_sim_pixels_with_mask_1 = flattened_sim_nsb10data[nsb10data_tuned_sim_masks_flat == 1]
# nsb10data_tuned_sim_pixels_with_mask_0 = flattened_sim_nsb10data[nsb10data_tuned_sim_masks_flat == 0] 

plt.figure(figsize=(16,11))

usefull_func.hist_wo_outliers(
    data_pixels_with_mask_1, 
    label = 'Data', 
    density=True, 
    mean_line=False, 
    color = colors[0], 
    alpha = 0.6
)
usefull_func.hist_wo_outliers(
    nsb_tuned_sim_pixels_with_mask_1, 
    label = 'Simulations with Tuned NSB', 
    density=True, 
    mean_line=False, 
    color = colors[1], 
    alpha = 0.4
)
usefull_func.hist_wo_outliers(
    global_sim_pixels_with_mask_1, 
    label = 'Simulations With NSB 0.24GHz', 
    density=True, 
    mean_line=False, 
    color = colors[2], 
    alpha = 0.4
)

# usefull_func.hist_wo_outliers(
#     nsb3data_tuned_sim_pixels_with_mask_1, 
#     label = 'Simulations With 3x NSB', 
#     density=True, 
#     mean_line=False, 
#     show_outliers=True, 
#     outlier_value=30, 
#     bin_number=1855, 
#     color = colors[3], 
#     alpha = 0.6
# )


# usefull_func.hist_wo_outliers(
#     nsb10data_tuned_sim_pixels_with_mask_1, 
#     label = 'Simulations With 10x NSB', 
#     density=True, 
#     mean_line=False, 
#     show_outliers=True, 
#     outlier_value=30, 
#     bin_number=1855, 
#     color = colors[4], 
#     alpha = 0.6
# )

plt.xlabel('Charge per pixel', fontsize = 14)
plt.ylabel('Probability density', fontsize = 14)
plt.ylim(0,0.5)
#plt.title("Charge per pixel only for pixels with signal")
plt.legend(prop={'size': 14})
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/dl1_hist_pixel_charge_signal.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,12))

usefull_func.hist_wo_outliers(
    data_pixels_with_mask_0, 
    label = 'Run17043', 
    density=True,  
    color = 'lightblue',
    bins = 500,
    alpha = 1,
    )

# usefull_func.hist_wo_outliers(
#     global_sim_pixels_with_mask_0, 
#     label = 'Simulation with 0.24GHz NSB (standard value)', 
#     density=True, 
#     mean_line=False, 
#     color = colors[6], 
#     alpha = 0.5,
#     bins = 500,
#     histtype = 'step',
#     linewidth = 3
# )

usefull_func.hist_wo_outliers(
    nsb_tuned_sim_pixels_with_mask_0, 
    label = 'Simulations with NSB tuned on the waveforms', 
    density=True, 
    mean_line=False, 
    color = 'green', 
    alpha = 1,
    histtype = 'step',
    linewidth = 3,
    bins = 500
)

# usefull_func.hist_wo_outliers(
#     nsbDL1data_tuned_sim_pixels_with_mask_0, 
#     label = 'Simulations with NSB tuned on the DL1 level', 
#     density=True, 
#     mean_line=False, 
#     show_outliers=True, 
#     outlier_value=30, 
#     bin_number=1855, 
#     color = colors[3], 
#     alpha = 0.5
# )

usefull_func.hist_wo_outliers(
    data_proper_nsb_tune_pixels_with_mask_0, 
    label = 'Simulations with NSB tuned with proper nominal value', 
    density=True, 
    mean_line=False, 
    color = 'red', 
    alpha = 0.5,
    bins = 500,
    histtype = 'step',
    linewidth = 3
)

# usefull_func.hist_wo_outliers(
#     data_high_nsb_pixels_with_mask_0, 
#     label = 'Data with high nsb', 
#     density=True, 
#     mean_line=False, 
#     show_outliers=True, 
#     outlier_value=30, 
#     bin_number=1855, 
#     color = colors[4], 
#     alpha = 0.3
# )

# usefull_func.hist_wo_outliers(
#     data_subrun_pixels_with_mask_0, 
#     label = 'Only one subrun for which were tuning performed', 
#     density=True, 
#     mean_line=False, 
#     show_outliers=True, 
#     outlier_value=30, 
#     bin_number=1855, 
#     color = colors[4], 
#     alpha = 0.6
# )

# usefull_func.hist_wo_outliers(
#     nsb_tuned_local_pixels_with_mask_0, 
#     label = 'Tuned local peak window sum', 
#     density=True, 
#     mean_line=False, 
#     color = colors[4], 
#     alpha = 0.6,
#     bins = 500
# )

# usefull_func.hist_wo_outliers(
#     local_sim_pixels_with_mask_0, 
#     label = 'Not tuned local peak', 
#     density=True, 
#     mean_line=False, 
#     color = colors[6], 
#     alpha = 0.6,
#     bins = 500
# )

plt.xlabel('Charge per pixel', fontsize = 14)
plt.ylabel('Probability density', fontsize = 14)
plt.ylim(0,0.45)
plt.xlim(-6,30)
#plt.title("Charge per pixel for pixels without signal")
plt.legend(prop={'size': 15})
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/dl1_hist_pixel_charge_no_signal_nsbtuned_clean.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
# Plot the histogram
data = data_pixels_with_mask_0
# Plot the histogram
count, bins, ignored = plt.hist(data, bins=30, density=True, alpha=0.6, color='g')

# Fit a normal distribution to the data
mu, std = stats.norm.fit(data)

# Plot the PDF
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
plt.title(title)

# Show plot
plt.show()

# Perform a goodness-of-fit test (Kolmogorov-Smirnov test)
ks_stat, p_value = stats.kstest(data, 'norm', args=(mu, std))
print(f"KS Statistic: {ks_stat}, P-value: {p_value}")

# Interpretation of p-value
if p_value > 0.05:
    print("The data follows a normal distribution (fail to reject H0)")
else:
    print("The data does not follow a normal distribution (reject H0)")

In [ ]:
# Plot the histogram
data = nsb_tuned_sim_pixels_with_mask_0

# Plot the histogram
count, bins, ignored = plt.hist(data, bins=30, density=True, alpha=0.6, color='g')

# Fit a normal distribution to the data
mu, std = stats.norm.fit(data)

# Plot the PDF
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
plt.title(title)

# Show plot
plt.show()

# Perform a goodness-of-fit test (Kolmogorov-Smirnov test)
ks_stat, p_value = stats.kstest(data, 'norm', args=(mu, std))
print(f"KS Statistic: {ks_stat}, P-value: {p_value}")

# Interpretation of p-value
if p_value > 0.05:
    print("The data follows a normal distribution (fail to reject H0)")
else:
    print("The data does not follow a normal distribution (reject H0)")

In [ ]:
sum(data_pixels_with_mask_0)

In [ ]:
sum(nsb_tuned_sim_pixels_with_mask_0)

### Second cleaning to conclude

In [ ]:
mask = tailcuts_clean(camgeom, data_dl1_images[2], picture_thresh=3, boundary_thresh=3)
fig, ax = plt.subplots(1, 2, figsize=(18, 10))
disp = CameraDisplay(
    camgeom, image=data_dl1_images[2], cmap="gray", ax = ax[0], title="Pixels with signal"
)
disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")
disp = CameraDisplay(
    camgeom, image=data_dl1_images[2], cmap="gray", ax = ax[1], title="Pixels without signal"
)
disp.highlight_pixels(~mask, alpha=0.8, linewidth=2, color="red")

In [ ]:
for test_image in data_dl1_images[:10]:
    mask = tailcuts_clean(camgeom, test_image, picture_thresh=10, boundary_thresh=5)
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[0], title="Pixels with signal"
    )
    disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[1], title="Pixels without signal"
    )
    disp.highlight_pixels(~mask, alpha=0.8, linewidth=2, color="green")

plt.close()

In [ ]:
for test_image in global_sim_dl1_image[:10]:
    mask = tailcuts_clean(camgeom, test_image, picture_thresh=10, boundary_thresh=5)
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[0], title="Pixels with signal"
    )
    disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[1], title="Pixels without signal"
    )
    disp.highlight_pixels(~mask, alpha=0.8, linewidth=2, color="green")

##

In [ ]:
for test_image in nsb1data_tuned_sim_dl1_image[:10]:
    mask = tailcuts_clean(camgeom, test_image, picture_thresh=10, boundary_thresh=5)
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[0], title="Pixels with signal"
    )
    disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")
    disp = CameraDisplay(
        camgeom, image=test_image, cmap="gray", ax = ax[1], title="Pixels without signal"
    )
    disp.highlight_pixels(~mask, alpha=0.8, linewidth=2, color="green")

In [ ]:
disp = CameraDisplay(
     camgeom, image=nsb1data_tuned_sim_dl1_image[2], cmap="gray", title="Image mask in green"
 )
mask = tailcuts_clean(camgeom, nsb1data_tuned_sim_dl1_image[0], picture_thresh=8, boundary_thresh=4)
disp.highlight_pixels(new_sim_mask[2], alpha=0.8, linewidth=2, color="green")

In [ ]:
max_limit = 6
fig, ax = plt.subplots(1, 2, figsize=(18, 10))
camgeom = source.subarray.tel[1].camera.geometry
#title="CT{}, run {} cog {} fit {} width {}".format(1,event.index.obs_id, round(event.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
disp = CameraDisplay(
    camgeom, image=real_data_proper_nsb_tune_dl1_image[2], cmap=plt.cm.RdBu_r, ax = ax[0], title="dl1 image sim"
)
#disp.set_limits_percent(20)
disp.set_limits_minmax(0,max_limit)
disp.add_colorbar()
disp = CameraDisplay(
    camgeom, image=data_dl1_images[2], cmap=plt.cm.RdBu_r, ax = ax[1], title="dl1 image data"
)

disp.add_colorbar()
disp.set_limits_minmax(0,max_limit)
#plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
plt.show()

In [ ]:
print(f"Mean number of pixels with signal for simulations GlobalPeak {sum(sum(global_sim_dl1_image_mask))/len(global_sim_dl1_image_mask)}")
print(f"Mean number of pixels with signal for simulations NSB tuned {sum(sum(nsb_tuned_sim_dl1_image_mask))/len(nsb_tuned_sim_dl1_image_mask)}")
print(f"Mean number of pixels with signal for data {sum(sum(data_dl1_mask))/len(data_dl1_mask)}")

print(f"Mean number of pixels without signal for simulations GlobalPeak {(len(global_sim_dl1_image_mask)-sum(sum(global_sim_dl1_image_mask)))/len(global_sim_dl1_image_mask)}")
print(f"Mean number of pixels without signal for simulations NSB tuned {(len(nsb_tuned_sim_dl1_image_mask)-sum(sum(nsb_tuned_sim_dl1_image_mask)))/len(nsb_tuned_sim_dl1_image_mask)}")
print(f"Mean number of pixels without signal for data {sum(sum(data_dl1_mask))/len(data_dl1_mask)}")

### Peak time

In [ ]:
flattened_sim_local = np.array(local_sim_dl1_peak_time).flatten()
flattened_sim_global = np.array(global_sim_dl1_peak_time).flatten()
flattened_sim_nsb = np.array(nsb_tuned_sim_dl1_peak_time).flatten()
flattened_data = np.array(data_dl1_peak).flatten()

plt.figure(figsize=(12,9))

usefull_func.hist_wo_outliers(flattened_data, label = 'Data', density=True, mean_line=False, show_outliers=True, outlier_value=100, bin_number=1855, alpha = 0.6)
usefull_func.hist_wo_outliers(flattened_sim_global, label = 'Simulations Global Peak', density=True, mean_line=False, show_outliers=True, outlier_value=100, bin_number=1855, alpha = 0.6)
usefull_func.hist_wo_outliers(flattened_sim_nsb, label = 'Simulations NSB tuned', density=True, mean_line=False, show_outliers=True, outlier_value=100, bin_number=1855, alpha = 0.6)

plt.xlabel('Peak time')
plt.ylabel('Probability density')
plt.ylim(0,0.5)
#plt.title("Peak time")
plt.legend()
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/dl1_hist_peak_time.pdf', dpi=200, format='pdf', bbox_inches='tight')


In [ ]:
np.nanmax(flattened_data)

In [ ]:
df_real_data.columns

## Distributions comparison

In [ ]:
bin_number = 8
alpha_reg = 0.7
plt.figure(figsize = (7,7))

sns.regplot(x = df_real_data_stacked['ring_radius'], y = df_real_data_stacked['ring_width'], x_bins=bin_number, color='k', label = 'Data', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_global['ring_radius'], y = df_sim_data_global['ring_width'], x_bins=bin_number, color=colors[2], label = 'Simulations with 100% optical efficiency', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_reflection_08624['ring_radius'], y = df_sim_data_reflection_08624['ring_width'], x_bins=bin_number, color=colors[4], label = 'Simulations with 110% optical efficiency', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_reflection_09408['ring_radius'], y = df_sim_data_reflection_09408['ring_width'], x_bins=bin_number, color=colors[3], label = 'Simulations with 120% optical efficiency', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_reflection_09016['ring_radius'], y = df_sim_data_reflection_09016['ring_width'], x_bins=bin_number, color=colors[3], label = 'Simulations with 120% optical efficiency', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})


plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring width [deg]')
plt.grid(linestyle="dashed")
plt.ylim(0,0.14)
plt.xlim(1.0,1.25)
plt.legend()
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/mcdata_width_reflections.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
bin_number = 10
alpha_reg = 0.7

plt.figure(figsize = (7,7))

sns.regplot(x = df_real_data_stacked['ring_radius'], y = df_real_data_stacked['ring_size'], x_bins=bin_number, color='k', label = 'Run 17043', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_global['ring_radius'], y = df_sim_data_global['ring_size'], x_bins=bin_number, color=colors[2], label = 'Simulations with nominal mirror reflection = 0.764', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_reflection_08624['ring_radius'], y = df_sim_data_reflection_08624['ring_size'], x_bins=bin_number, color=colors[4], label = 'Simulations with 110% of nominal mirror reflection', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_reflection_09408['ring_radius'], y = df_sim_data_reflection_09408['ring_size'], x_bins=bin_number, color=colors[3], label = 'Simulations with 110% of nominal mirror reflection', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_reflection_09016['ring_radius'], y = df_sim_data_reflection_09016['ring_size'], x_bins=bin_number, color=colors[5], label = 'Simulations with 110% of nominal mirror reflection', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})


plt.xlabel('Muon radius [deg]', fontsize=15)
plt.ylabel('Muon ring size [pe]', fontsize=15)
plt.grid(linestyle="dashed")
plt.ylim(1500,3500)
plt.xlim(1.0,1.23)
plt.legend()
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/mcdata_size_reflections.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
print(sorted(muon_files))

df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[4:]] 
df_all_data = pd.concat(df_files, ignore_index=True)
df_good_data_real = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['size_outside'] < 500)]
del(df_all_data)
df_good_data_real

In [ ]:
from datetime import datetime
start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

In [ ]:
bin_number = 10
alpha_reg = 0.7
size_outside = 500

plt.figure(figsize = (11,11))

#sns.regplot(x = df_good_data_real['ring_radius'], y = df_good_data_real['ring_size'], x_bins=bin_number, color=colors[0], label = 'Data', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})


sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
                                                 (df_good_data_real['ring_containment'] > ring_containment_cut) & 
                                                 (df_good_data_real['size_outside'] < size_outside) &  
                                                (df_good_data_real['event_time'] > start_date_2023) &
                                                (df_good_data_real['event_time'] < end_date_2023)], 
            y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
                                               (df_good_data_real['ring_containment'] > ring_containment_cut) &  
                                               (df_good_data_real['size_outside'] < size_outside) & 
                                                (df_good_data_real['event_time'] > start_date_2023) &
                                                (df_good_data_real['event_time'] < end_date_2023)], 
            x_bins=bin_number, color=colors[0], label = 'Data for 2023 year')
sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
                                                 (df_good_data_real['ring_containment'] > ring_containment_cut) & 
                                                 (df_good_data_real['size_outside'] < size_outside) &  
                                                (df_good_data_real['event_time'] > start_date_2024) &
                                                (df_good_data_real['event_time'] < end_date_2024)], 
            y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
                                               (df_good_data_real['ring_containment'] > ring_containment_cut) &  
                                               (df_good_data_real['size_outside'] < size_outside) & 
                                                (df_good_data_real['event_time'] > start_date_2024) &
                                                (df_good_data_real['event_time'] < end_date_2024)], 
            x_bins=bin_number, color=colors[1], label = 'Data for 2024 year')

sns.regplot(x = df_sim_data_global['ring_radius'], y = df_sim_data_global['ring_size'], x_bins=bin_number, color='k', label = 'Muon simulations processed with GlobalPeakWindowSum', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_reflection_08624['ring_radius'], y = df_sim_data_reflection_08624['ring_size'], x_bins=bin_number, color=colors[4], label = 'Simulations with 110% of nominal mirror reflection', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_reflection_09408['ring_radius'], y = df_sim_data_reflection_09408['ring_size'], x_bins=bin_number, color=colors[3], label = 'Simulations with 120% of nominal mirror reflection', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_local['ring_radius'], y = df_sim_data_local['ring_size'], x_bins=bin_number, color='k', label = 'Muon simulations processed with LocalPeakWindowSum (default image extractor)', scatter_kws={'alpha': alpha_reg}, line_kws={'linestyle': '--'})

plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [pe]')
plt.grid(linestyle="dashed")
plt.ylim(1500,3500)
plt.xlim(1.0,1.25)
plt.legend(prop={'size': 10})
plt.savefig('/Users/vdk/muons2024/images/PlanPaper/mcdata_size_2023-24year_allreflections.pdf', dpi=200, format='pdf', bbox_inches='tight')